In [ ]:
import numpy as np
import pandas as pd
import xgboost as xgb
import pickle as pickle
import nltk

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from gensim.models.word2vec import Word2Vec
from sklearn import preprocessing,metrics 
pd.set_option("display.max_columns", 100)
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn import linear_model,svm
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.feature_extraction import FeatureHasher
from sklearn.metrics import average_precision_score, classification_report,confusion_matrix
from scipy.stats import skew
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from tqdm import tqdm
from nltk import word_tokenize
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import accuracy_score

In [ ]:
classes = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
data = pd.read_csv('train.csv').fillna(' ')
data['comment_text'].fillna("missing", inplace=True)
X_train, X_test, y_train, y_test = train_test_split(data["comment_text"],data[classes],test_size=0.25,random_state=55)
data_text = pd.concat([X_train,X_test])

In [ ]:
#glove embedding
GLOVE_6B_50D_PATH = "glove.6B.50d.txt"
glove = {}
glove_path = open(GLOVE_6B_50D_PATH)
for line in tqdm(glove_path):
    line_arr = line.split()
    word = line_arr[0]
    glove[word] = np.asarray(line_arr[1:], dtype='float32')
glove_path.close()

In [ ]:
#normalization 
nltk.download('punkt')
porter = PorterStemmer()
sw = set(stopwords.words('english'))
sw.update(['.', "'", '(', ')', '[', ']', '{', '}'',', '"', ':', ';'])
def vectorize(s):
    words = str(sentence).lower()
    tokenized = word_tokenize(words)
    updated_tokens=[]
    final_tokens = []
    for word in tokenized:
        if not word in sw and word.isalpha(): 
            updated_tokens.append(word)
    for token in updated_tokens:
        try:
            final_tokens.append(glove[token])
        except:
            continue  
    final_tokens = np.array(final_tokens)
    total =  final_tokens.sum(axis=0)
    data_type = type(total)
    if data_type != np.ndarray: return np.zeros(60)
    denom = np.sqrt((total ** 2).sum())
    return total / denom

In [ ]:
xg_train = [vectorize(sentence) for sentence in tqdm(X_train)]
xg_test = [vectorize(sentence) for sentence in tqdm(X_test)]

In [ ]:
#logistic regression
logreg = OneVsRestClassifier(LogisticRegression())
logreg.fit(xg_train, y_train)
y_pred = logreg.predict_proba(xg_test)
accuracy = accuracy_score(y_test, y_pred, normalize=False)
print(" Test Accuracy: %.2f%%" % (accuracy * 100.0))

In [ ]:
#svm 
currsvm = OneVsRestClassifier(SVC())
currsvm.fit(xg_train, y_train)
y_pred = currsvm.predict_proba(xg_test)
accuracy = accuracy_score(y_test, y_pred, normalize=False)
print(" Test Accuracy: %.2f%%" % (accuracy * 100.0))

In [ ]:
#xgb
xgbmodel = OneVsRestClassifier(xgb.XGBClassifier(max_depth=4, n_estimators=100,learning_rate=0.1, silent=False))
xgbmodel.fit(xg_train, y_train)
y_pred = clf.predict_proba((xg_test)
accuracy = accuracy_score(y_test, y_pred, normalize=False)
print(" Test Accuracy: %.2f%%" % (accuracy * 100.0))                            